# This tutorial is based on Deeplearning prompt engineering crash course


In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

# Part 1: Important Guidelines

## First Principle

First prinnciple of writing good prompts is to write is clear and precise.It does not matter how long the prompt is if it able to clearly specify the task.Following tactics can be followed to write such prompts.


### Use Delimeter and punctuation to separate different sections of text

In [ ]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

#In this example I tried by removing some delimeters and making the prompt less structured.
#But it does not seemed to have major effect.But still it can be assumed 
#that better answer will be given for the prompt which follows this tactics.

### this tactic is development based.As we can ask the ChatGPT to also return the output in particular format.This feature provides us ease of processing a standard format output.

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

### We can also ask the ChatGPT to provide condition based answers.We can multiple conditions and thier responses in the promp.One use of this type of feature is responding the user on the basis of emotion predicted in the input text.

In [ ]:
text_2 =
f"""
Saurav is smart and intelligent.
""" 

prompt =
f"""
You will be provided with text delimited by triple backticks. 

If the text contains only good things about Saurav.Then write the following response: \
\"Thanks,You are good guy. \"

-------------------------------------
If the text indicate that the writer is jealous from Saurav.Then write the following response: \
\"It seems that you are jealous of Saurav. \"

--------------------------------------
If the text contains at least one thing bad about Saurav and it does not indicate jealousy.Then write the following response: \
\"You are `emotion` of Saurav.\".Here `emotion' is a place holder for the emotion depicted in the text.

---------------------------------------
The text is: \
```{text_2}```

"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

#it was a funny way of checking this feature.* - *

### Few Shot Prompting.This feature is based on making the model understand the output structure and then reply to the input.There are some use cases that I can think of this feature:

* Solving Puzzles
* Restructuring Structured Data

In [ ]:
prompt = f"""
You will be given some example conatining the question and thier respective answer. \
Through these examples, you need to understand the reason behind each answer. \
Each answer will be based upon same reasoning. \
Your task is to give the answer of a question using the same reasoning that was appliied while solving the example questions.

 ---------------------
 Examples:
 1. Question: Apple+Mango
    Answer: Orange
 2. Question: Blueberry+Pineberry
    Answer: Skyblue
 3. Question: Blueberry+Mango
    Answer:   Green

 Question to be answered:
 Question: Apple+Green Grapes
 Answer: ?
 
"""

response = get_completion(prompt)
print(response)

##In this example model is not able to understand the reasoning clearly.The answer given by Model is "Red".But the actual answer is orange.

## Principal 2: Give the model time to think

### In case if the problem is little complex to solve,then we can change the prompt using following tactics.These tactics increase the amount of computation but also helps in increasing accuracy:

* Specify the steps that model should follow to get the response

* Ask the model to work out its own solution first

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

#the answer in this case was wrong,hence we will follow the second point

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

#in this case answer came out be correct.Let's try this method in our own problem.

In [ ]:
##lets try some points from above prompt

prompt = f"""

Your task is to give the solution of the question:
To solve the problem do the following:
- First, work out your own solution to the questions whoose answer is given
- Then compare your solutions to the given answers
and evaluate if your solutions is same as given answers. 
Don't decide on the solution of unanswered question until all the answers of the questions whoose answer is given \
matched to your answers.

-----------------------
Use the following format:
Question:
```
question here
```
Answer:
```
answer here
```

 ---------------------
Question
```
You need to give correct answer of the question where answer is written as `?`.All the answer is based on some specific reasoning.\
You can deduce that reasoning from the questions for which answers are available.And aplly same reasoning to the other questions that needed to be answered.
 1. Question: Apple+Mango
    Answer: Orange
 2. Question: Blueberry+Pineberry
    Answer: Skyblue
 3. Question: Blueberry+Mango
    Answer:   Green
 4.  Question: Apple+Green Grapes
     Answer: ?
```

"""

response = get_completion(prompt)
print(response)

##Here also model failed.Hmm,do now know where the problem is.Might try another variation later.

## Model Hallucination

In [ ]:
#sometime the model can create information that is not actually present
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

#here Boie is real but there is not product like AeroGlide UltraSlim Samrt Toothbrush

In [ ]:
#there is one way to reduce this problem,
#is to ask the model to find the relevant information and then answer based on that relevant information
prompt = f"""

```
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie.
```

While answering this question firstly find relevant information.\
Then answer the question based on the relevant information
"""

response = get_completion(prompt)
print(response)

#here the answer comes out to be correct

## Let's explore the summary building capability of ChatGPT

### let's build a review reader for a ecommerce site that can have following features:
* It should be able to summarise the review based on the following context filters:
    * Product
    * Pricing
    * Shipping
* It should be able to summarise the review to the number of words inputted by user.
* In the summary it should be able to put more focus on a particular context.The context can be the following and will be inputted by user
    * Product
    * Pricing
    * Shipping

In [2]:
#define some custom data types
from enum import Enum
from typing import Any

# for context
class Context(Enum):
    PRODUCT="PRODUCT"
    PRICING="PRICING"
    SHIPPING="SHIPPING"
    
class SummaryType(Enum):
    CONTEXTRELEVANT="RELEVANT TO CONTEXT"
    FULLSUMMARY="FULL SUMMARY"

# a data type that is used to limiting the number of words 
class LimitNumber:
    def __init__(self, value):
        if not isinstance(value, int) or value <= 0:
            raise ValueError("Value must be a positive integer.")
        self.value = value
    
        
class ContextReviewPrompts():
    
    PRODUCT_REVIEW_PROMPT="""to give feedback to the \
    product deparmtment, responsible for overseeing merchandise in online \
    business operations and strategies.  \

    Summarize the review below, delimited by triple \
    backticks,by focusing on any aspects \
    that are relevant to the product quality."""
    
    SHIPPING_REVIEW_PROMPT="""to give feedback to the \
    shipping deparmtment. \

    Summarize the review below, delimited by triple \
    backticks, by focusing on any aspects \
    that are relevant to the shipping and delivery"""
    
    PRICING_REVIEW_PROMPT="""
    to give feedback to the \
    pricing deparmtment, responsible for determining the \
    price of the product.  \

    Summarize the review below, delimited by triple \
    backticks, by focusing on any aspects \
    that are relevant to the price and perceived value.
    """
    
    def __call__(self, context: Context) -> Any:
        
        if context == Context.PRODUCT:
            return self.PRODUCT_REVIEW_PROMPT
        
        if context == Context.PRICING:
            return self.PRICING_REVIEW_PROMPT
        
        if context == Context.SHIPPING:
            return self.SHIPPING_REVIEW_PROMPT  
        
class SummarySpecificPrompts():
    
    CONTEXTRELEVANT="to extract relevant information from"
    FULLSUMMARY="to generate a short summary from"
    
    def __call__(self,summary_type: SummaryType):
        
        if summary_type == SummaryType.FULLSUMMARY:
            return self.FULLSUMMARY
        
        if summary_type == SummaryType.CONTEXTRELEVANT:
            return self.CONTEXTRELEVANT
        

In [7]:
#utils functions;

def get_context_prompt(context: Context=Context.PRODUCT):
    
    context_based_prompt=ContextReviewPrompts()
    return context_based_prompt(context = context)

def get_summary_type_prompt(summary_type: SummaryType = SummaryType.FULLSUMMARY):
    summary_type_prompts=SummarySpecificPrompts()
    return summary_type_prompts(summary_type = summary_type)

def get_final_prompt(review,context: Context = Context.PRODUCT, summary_type : SummaryType = SummaryType.FULLSUMMARY, \
                    total_words : LimitNumber = None ):
    
    summary_prompt=get_summary_type_prompt(summary_type)
    context_prompt=get_context_prompt(context)
    
    prompt= f"""Your task is {summary_prompt} a product \
    review from an ecommerce site {context_prompt} \

    Review: ```{review}```
    """
    
    if total_words:
        limit_to_words=total_words.value
        prompt=f"""Your task is {summary_prompt} from a product \
        review from an ecommerce site {context_prompt} \
        
        Limit to {limit_to_words} words. \
        
        Review: ```{review}``` 
        """
        
    return prompt
                

In [8]:
review= """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""
context="SHIPPING"
summary_type="FULLSUMMARY"
limit_to_words=LimitNumber(10)
# limit_to_words=None

prompt=get_final_prompt(review,context = Context[context], summary_type = SummaryType[summary_type],total_words=limit_to_words)

get_completion(prompt)

'Your task is to generate a short summary of a product         review from an ecommerce site to give feedback to the     product deparmtment, responsible for overseeing merchandise in online     business operations and strategies.  \n    Summarize the review below, delimited by triple     backticks,by focusing on any aspects     that are relevant to the product quality.         \n        Limit to 10 words.         \n        Review: `````` \n        '

## We can also do following ML based tasks with ChatGPT:
* Sentiment Analysis
* Emotion Prediction
* Topic Prediction
* Label Extraction (Extraction particular labels like company name,product name from text)

In [9]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast. But the string was not working.Asked for replacement.Got the new one in three days.
The new one also had an issue in thier lamp.Thier product qualtity is too bad.The customer service is also not good.
"""

In [ ]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
#we can also ask the model for confidence value of each emotion prediction
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list.

Also for which emotion, give the confidence value indicating your confidence regarding presence of that particular emotion.

Please give answer in following format
```
Emotions: List of emotions here
Confidence: List of confidence value correspoing to each emotion present in Emotions.
```
Confidence value should be between 0 and 1.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
# Predict topics of a text
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

In [10]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [ ]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

### Some Important Findings
* Let suppose in a text that has positive sentiment,models was asked to give the negative emotions,then model will inlcude some postive emotions in the list.
* Hence in this case we should follow the principal of giving the model time to think.
* We can ask the model to follow following steps:
    * Predict the Sentiment
    * If positive,list down the positive emotions
    * If negative,do nothing.

## Transforming capapbilities of ChatGPT

We can use ChatGPT to do the following things:
* Translation from one lanuage to another lanuage.
* Translation or conversion of a text into different tunes.For example converting a piece of text into formal or informal tunes.
* It can be used as a Universal Translator!!!
* It can be used to grammar correction.
* Convert one format of piece of text to another format.For example JSON to HTML or python to JAVA.

Let's explore each one of the example one by one

In [ ]:
# translation
prompt = f"""
Translate the following spanish text to Sanskrit: \ 
```Hola, me gustaría ordenar una licuadora.```
"""
response = get_completion(prompt)
print(response)

#NOTE: 
    #   If instead of spanish text,I use only the text in the prompt,then model results quanlity decreases.
    # We can also ask the model to write answer in multiple tunes like formal or informal

In [ ]:
# Universal Translator
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

In [ ]:
#Tone transformation
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

In [ ]:
# transform the type (json to HTML).
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table: {data_json}
"""
response = get_completion(prompt)
print(response)

#display the html response
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

In [ ]:
#spell check or grammer check
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    any errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

## Expansion capability of model

We can ask model to use a piece of text to generate something else.For example,we can generate an AI generated reply mail from a user review.

In [1]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [ ]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt,temperatur=0.7)
print(response)

### Temperature parameter
Temperatur parameter is used to introduce randomness in the response of model.If temperature is zero,then model always picks the next word which has best confidence.For example,After "My Best Food is",it will always picks up pizza as pizza has the highest probability if the temperature is zero.But if temperature is not zero,then it might also pick up other words like tacos,burger etc.This randomness increases as we increase the temperature.So here are following tips for using the temperature parameter.

* If we want the output to be predictable.For example during transformation use cases,we can keep the temperature to be zero.
* If we want the output to be more creative and random,for example in chatbot or a poem generation,we can should increase the temperature.



## ChatBOT

### Let firstly understand how does the model works internally while using the ChatGPT web interface.

So, in general ChatGPT is a chat based model where a user give a input to a assistant and assistant replies to that input.

                Assistant
                 ↑  ↓
                 User

Every new call to the model is a new conversation,so to let the model gives answers based on previous conversation,we need to give the previous conversation along with the current query as an input to the model.The previous conversation is called "context" for the model.

Now before starting a conversation with the model,we can also specify how does the model should behave during conversation.So for a chatbot,there are generaly three type of prompts in a conversation.

1. system: specifies how does the models should behaves.For example,we can ask the model to be a friendly assistant and answer like Shakespeare.

2. assistant : The message by the model.

3. user : The message by the user.

                   System 
                      ↓ (only once in a conversion)
                   Assistant
                    ↑  ↓  (repeated conversation)
                    User              

In [ ]:
# lets try this practically
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},  #sets the assistant behaviour in this conversation  
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response) #will give response according to previous conversation and the behaviour specified by the system.

## JasonBOT

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are an JasonBot.A chatbot that can give answers to questions about Jason.
Only give answer if you can found the exact answer in the given information about the Jason.Do not create your own answer.If answer is not found then give the following answer 
```
Sorry,I do not know.
```

Please use the following information about Jason to answer any question asked to you.
```
Name : Jason Derulo
Age : 24
Color : Brown
Relationship Status : Not single
Girlfriend Name : Nora Fatehi
Mother Name : Angelina
Father Name : Brad
Sister Name : Kim
Best Food : Paneer Curry
Best Sports: Football
Education : Graduate
Alma Matter : Stanford
Job Satus : AI Engineer at OpenAI
Other features : Shy,Smart,Intelligent,Healthy
Weight : 65kg
```
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard